# Perspective spaces

## Basic setup

Utils

In [1]:
from PIL import Image, ImageFont


class Depiction:
    def __init__(self, layer, style, subject, img_src):
        self.layer = layer
        self.style = style
        self.subject = subject
        
        self.resize_ratio = 0
        self.x = None
        self.y = None
        self.w = None
        self.h = None
        
        self.img_src = img_src
        self.img = Image.open(img_src)
        
    def __str__(self):
        return "{} {} (layer {})".format(self.style, self.subject, self.layer)
    
    def __repr__(self):
        return "{} {} (layer {})".format(self.style, self.subject, self.layer)
    
    def update_size(self, w, h):
        self.w = w
        self.h = h
        self.img = self.img.resize((w, h))

In [2]:
def draw_rect(img, pos1, pos2, color="grey"):
    x1, y1 = pos1  
    x2, y2 = pos2
    img_border = ImageDraw.Draw(img) 
    shape = [(x1, y1), (x2, y2)]
    img_border.rectangle(shape, outline=color, width=80)
    
def draw_text(img, pos, text):
    x, y = pos
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype("Gidole-Regular.ttf", size=100)
    draw.text((x, y), text, font = font, align ="left", fill="black") 
    
def is_collided(rect1, rect2):
    x1, y1, w1, h1 = rect1
    x2, y2, w2, h2 = rect2
    
    c1 = np.array([x1 + w1/2, y1 + h1/2])
    c2 = np.array([x2 + w2/2, y2 + h2/2])

    dx = np.abs(c1 - c2)[0]
    dy = np.abs(c1 - c2)[1]
        
    return dx < (w1/2 + w2/2) and dy < (h1/2 + h2/2)

In [3]:
def save_img(img, fpath, save_ratio=1):
    w, h = img.size
    resized = img.resize((int(w*save_ratio), int(h*save_ratio)))
    resized.save(fpath)

Load depictions from image files

In [4]:
# load all images
import os


depictions = []

for dir, subdirs, files in os.walk('./'):
    if "output" in dir: 
        continue
    if not subdirs and not dir.startswith("./."):
        for file in files:
            # print(os.path.join(dir, file))
            layer = int(dir[-1])
            style = file.replace(".png", "").split(",")[-1].replace("art", "").strip()
            
            if "ukiyo-e" in file:
                subject = file.split("-")[-2].split(",")[0].strip()
            else:
                subject = file.split("-")[-1].split(",")[0].strip()

            depiction = Depiction(layer, style, subject, os.path.join(dir, file))
            depictions.append(depiction)

In [5]:
depictions = sorted(depictions, key=lambda x: x.layer, reverse=False)

In [6]:
len(depictions)

111

## Image creation

First, make an empty canvas

A1 in pixels is: 7,016 x 9,933

In [7]:
from PIL import Image, ImageDraw

WIDTH = 7016
HEIGHT = 9933

img = Image.new("RGB", (WIDTH, HEIGHT), (255, 255, 255))

### Add the layers

In [8]:
import random
from tqdm.auto import tqdm
import numpy as np


RANDOM_SEED = 5
RESIZE_FACTOR = 1.2
MIDDLE_LAYER = 2.5
SHIFT = 0.05
ZONE_PORTION = 0.25
PLACEMENT_RANDOM_PERTURB = 0
SAVE_RATIO = 1

WIDTH_INNER = 1770
HEIGHT_INNER = 1770

inner_center_x = WIDTH_INNER / 2
outer_center_x = WIDTH / 2
inner_center_y = HEIGHT_INNER / 2
outer_center_y = HEIGHT / 2
random.seed(RANDOM_SEED)

In [9]:
img_inner = Image.new("RGB", (WIDTH_INNER, HEIGHT_INNER), (255, 255, 255))


def draw_layer(base_img, layers):
    def place_depiction(image, depiction, zone):
        # assign random position in a center square
        center_size = 500
        x = int(WIDTH/2 + random.randint(-center_size, center_size)) #random.randint(0, WIDTH) #
        y = int(HEIGHT/2 + random.randint(-center_size, center_size)) #random.randint(0, HEIGHT) #

        # define the no-go zone
        zone_width = int(depiction.w * ZONE_PORTION) * zone
        zone_height = int(depiction.h * ZONE_PORTION) * zone

        no_go_zone_x, no_go_zone_y = WIDTH/2 - WIDTH_INNER/2 - zone_width, HEIGHT/2 - HEIGHT_INNER/2 - zone_height
        no_go_zone_w, no_go_zone_h = WIDTH_INNER + zone_width*2, HEIGHT_INNER + zone_height*2
        no_go_zone = (no_go_zone_x, no_go_zone_y, no_go_zone_w, no_go_zone_h)
#         draw_rect(image, (no_go_zone_x, no_go_zone_y), (no_go_zone_x+no_go_zone_w, no_go_zone_y+no_go_zone_h), "green")

        # push out from no-go zone in random directions with random speed 
        direction = random.uniform(-1, 1), random.uniform(-1, 1)
        speed = 300

        depiction_rect = (x,y, depiction.w, depiction.h)
        while is_collided(depiction_rect, no_go_zone):
            x += direction[0] * speed
            y += direction[1] * speed
            depiction_rect = (x,y, depiction.w, depiction.h)
            
        # once outside, perturb randomly for more natural look
        x += PLACEMENT_RANDOM_PERTURB * random.uniform(-depiction.w, depiction.h)
        y += PLACEMENT_RANDOM_PERTURB * random.uniform(-depiction.h, depiction.h)
        return x, y
    
    middle_frame = len(layers) // 2 + 1
    print("Middle frame:", middle_frame)
    
    inner_image = Image.new("RGB", (WIDTH_INNER, HEIGHT_INNER), (255, 255, 255))
    for current_frame, _ in enumerate(tqdm(layers)):
        image = base_img.copy()

        # draw up to the layer corresponding to current frame
        for layer_no, layer in enumerate(layers[:current_frame+1]):
            for i, depiction in enumerate(layer):
                if depiction.layer == 1:
                    continue
                    
                # resize
                if not depiction.w or not depiction.h:
                    
                    resize_ratio = (RESIZE_FACTOR*random.uniform(0.98, 1.02))**(depiction.layer - MIDDLE_LAYER)
                    depiction.resize_ratio = resize_ratio
                    orig_w, orig_h = int(WIDTH_INNER/3), int(HEIGHT_INNER/3)
                    depiction.update_size(int(orig_w * resize_ratio), int(orig_h * resize_ratio))
                    
                # position
                if not depiction.x or not depiction.y:
                    # if not previously placed, place 
                    x, y = place_depiction(image, depiction, depiction.layer - 2)
                    center_offset = (middle_frame - current_frame)* ((depiction.layer-MIDDLE_LAYER) * depiction.w * SHIFT)
                    depiction.x = x + center_offset
                    depiction.y = y
                else:
                    # else, apply parallax effect
                    depiction.x += (MIDDLE_LAYER-depiction.layer) * SHIFT * depiction.w 

                image.paste(depiction.img, (int(depiction.x), int(depiction.y)))
                # draw_text(img, (x,y), str(depiction))
            
            if layer_no <= middle_frame:
                if current_frame != 0:
                    layer_ones = [d for d in layers[1] if d.layer == 1 and d.subject != "dog"]
                    center_piece = [d for d in layers[1] if d.layer == 1 and d.subject == "dog"][0]

                    layer_ones.insert(4, center_piece)

                    for i, depiction in enumerate(layer_ones):
                        im = depiction.img
                        im = im.resize((int(WIDTH_INNER/3), int(HEIGHT_INNER/3)))
                        depiction.x = int(i % 3 * WIDTH_INNER/3)
                        depiction.y = int(i // 3 * HEIGHT_INNER/3)
                        inner_image.paste(im, (depiction.x, depiction.y))
                draw_rect(inner_image, (0,0), (WIDTH_INNER, HEIGHT_INNER), "red")
                image.paste(inner_image,
                        (int(outer_center_x - inner_center_x), 
                        int(outer_center_y - inner_center_y)))
        
        # Lastly, draw outer border and save
        draw_rect(image, (0,0), (WIDTH, HEIGHT), "#4472c4")
        container_dir = "output{}".format(RANDOM_SEED)
        for folder in [container_dir, container_dir + "/full_size", container_dir + "/small_size"]:
            if not os.path.isdir(folder):
                os.mkdir(folder)
        save_img(image, "output{}/full_size/{}.jpg".format(RANDOM_SEED, current_frame), save_ratio=SAVE_RATIO)
        save_img(image, "output{}/small_size/{}.jpg".format(RANDOM_SEED, current_frame), save_ratio=0.2)

In [10]:
layers = []
for i in range(11):
    layer = [d for d in depictions if d.layer == i]
    layers.append(layer)

In [ ]:
draw_layer(img, layers)

Middle frame: 6


  0%|          | 0/11 [00:00<?, ?it/s]